# Pset #2 part 2 of 2: pick and place using the iiwa arm


In this notebook, you'll be asked to use IK (inverse kinematics) and ICP (iterative closest point) to pick up an object and place it at a designated location. To get a quick idea of what needs to be accomplished, **click on the picture below to watch the full task on Youtube.**
<p align="center" style="text-align: center">
   Bleach bottle pick and place with iiwa. Point cloud is overlaid on visual geometries. The bottle is airdropped for dramatic effect.  
   <a href="https://youtu.be/w3jNxQ-8MWI"><img src="./resources/iiwa_picks_up_bottle.png" width="700"></a>
</p>

In [ ]:
# Import stuff to be used in this notebook. Run this first!
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import meshcat
import meshcat.geometry as g

from iterative_closest_point import icp, make_meshcat_color_array
from point_cloud_processing import (VisualizeTransformedScenePointCloud,
                                    SegmentBottleZfilter,
                                    FindBottlePointInCroppedScenePointCloud,
                                    SegmentBox,
                                    GetBoxModelPointCloud)                                   
from kuka_multibody_sim_w_control import (CreateMultiBodyPlant,
                                          RunSimulation)
from kuka_pick_and_place_trajectory_generation import GenerateIiwaTrajectoriesAndGripperSetPoints

In [ ]:
# First start meshcat for visualization - this only has to be run once.

# If you interrupt the kernel of this notebook, you'll need to run this cell again to 
# restart the meshcat server, and then refresh the visualization window. 

# This will open a mesh-cat server in the background, click on the url to display visualization in a separate window. 
vis = meshcat.Visualizer()

In [ ]:
# Run this if you feel the visualizer is messed up and want to start over.
vis.delete()

## 1. Perception tasks
You are given a point cloud of the scene at its initial condition. The point cloud is generated by four simulated (perfect) depth cameras. From this point cloud, you need to find
- the pose (position and orientation) of the bottle, and 
- the pose of the green box. 

You are provided with a point cloud of the scene, which includes the tables, the bottle, the box and the robot at its upright posture. The coordinates of the points in the point cloud are expressed relative to the world frame `W`.

Run the following cell to take a look at the scene point cloud in meshcat. Note that there are a lot of points on the floor (shown in blue). In practice, these points are helpful to align point clouds generated by multiple cameras. 

In [ ]:
# load scene point cloud
scene_point_cloud = np.load('./resources/scene_point_cloud.npy') 

# color point cloud bazed on height (z)
min_height = scene_point_cloud[:,2].min()
max_height = scene_point_cloud[:,2].max()*1.1
colors = cm.jet((scene_point_cloud.T[2, :] - min_height) / (max_height - min_height)).T[0:3, :]

# show point cloud in meshcat
vis.delete()
vis["scene_point_cloud"].set_object(
    g.PointCloud(position=scene_point_cloud.T,
                 color=colors,
                 size=0.002))

### 1.1 Estimating the pose of the bottle in the world frame using ICP. 
Let's first load the model point cloud. A red point cloud in the shape of the bottle will appear at the origin after running the cell below. Rotate the camera and zoom in to see it. 

In [ ]:
# Load point cloud of the bleach bottle.
bottle_point_cloud_model = np.load('./resources/bleach_bottle_point_cloud.npy')
n_model = bottle_point_cloud_model.shape[0]

# show point cloud in meshcat
vis['model_point_cloud'].set_object(
    g.PointCloud(bottle_point_cloud_model.T, make_meshcat_color_array(n_model, 0.5, 0, 0), size=0.002))

As the scene frame `S` is the same as the world frame `W`, if we can find the transform `X_MS`, then the pose of the model in the world frame can be found by inverting `X_MS`, i.e. `X_WM = X_MS^(-1)`.


However, running ICP between `model_point_cloud` and `scene_point_cloud` probably won't give meaningful results, because the majority of the points in `scene_point_cloud` are outliers, e.g. table/box/ground points. For the fun of it, you can run the following cell to see what happens if we do that. It's gonna take a while because there are quite a few points in the scene point cloud.

In [ ]:
# Call ICP to match full scene point cloud to model point cloud.
X_MS_bottle, mean_error, num_iters = \
    icp(scene_point_cloud, bottle_point_cloud_model, max_iterations=100, tolerance=1e-5)
print "X_MS_bottle:"
print X_MS_bottle
print "mean error:", mean_error
print "total iterations run:", num_iters

# Create a yellow meshcat point cloud for visualization.
VisualizeTransformedScenePointCloud(X_MS_bottle, scene_point_cloud, vis)

A natural way to resolve this issue is to run ICP with only the scene points that correspond to the object of interest. The process of cropping out the correct subset of `scene_point_cloud` that corresponds to the model is known as segmentation, which is still [an area of active research](https://arxiv.org/abs/1703.01661). Later in the course, we'll talk more about using other methods such as CNN to get initial estimates of object location and better segmentations. In this pset, however, we'll try segmentation with some heuristics. 


The most straightforward heuristics is perhaps the "z-filter". Assuming that we know the height of the table, then every point with a z-coordinate smaller than that can be discarded. We've written a function that implements the z-filter for you:

In [ ]:
from kuka_multibody_sim_w_control import table_top_z_in_world

# Segment bottle points from scene
scene_point_cloud_cropped = SegmentBottleZfilter(scene_point_cloud, z_threshold=table_top_z_in_world+0.001)

# Delete the messy yellow point cloud.
vis['transformed_scene'].delete()

# Replace the scene point cloud with the cropped version. 
vis["scene_point_cloud"].set_object(
    g.PointCloud(position=scene_point_cloud_cropped.T,
                 color=make_meshcat_color_array(scene_point_cloud_cropped.shape[0], 0, 0.0, 1.0),
                 size=0.002))

You've probably noticed that there are more than one objects on the table. We need the bottle, but not the box or the bottom of the robot.

**Your task** is to complete function `FindBottlePointInCroppedScenePointCloud` in `point_cloud_processing.py` to create a segmentation that allows ICP to converge to the correct pose. 

To segment the bottle from the z-filtered point cloud, you could use [k-means](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), which is simple to implement or some variant of "connected components" as described [here] (https://cs.gmu.edu/~kosecka/ICRA2013/spme13_trevor.pdf). K-means should work in this case but would have trouble if the objects were more complicated or near to each other.  

You can also assume we know that the bottle should be close to `bottle_center_initial_guess = [0.8, 0, table_top_z_in_world]` in world frame. The cluster whose mean is closest to that given point is the point cloud of the bottle. 

In [ ]:
from point_cloud_processing import bottle_center_initial_guess
indices_bottle = FindBottlePointInCroppedScenePointCloud(scene_point_cloud_cropped, bottle_center_initial_guess)

# Delete the messy yellow point cloud.
vis['transformed_scene'].delete()

# Replace scene point cloud with the segmented point cloud.
vis['scene_point_cloud'].set_object(
    g.PointCloud(scene_point_cloud_cropped[indices_bottle].T, 
                 make_meshcat_color_array(len(indices_bottle), 0., 0., 1.0), size=0.002))

Running ICP on the segmented point cloud should give a much better estimation of the bottle's pose. 

In [ ]:
# Call ICP to match bottle_points to model point cloud
X_MS_bottle, mean_error, num_iters = \
    icp(scene_point_cloud_cropped[indices_bottle], bottle_point_cloud_model, 
        init_guess = np.eye(4), max_iterations=100, tolerance=1e-8)
print "mean error:", mean_error
print "total iterations run:", num_iters

# compare ground truth of the bottle pose in world frame with the one returned by ICP
from kuka_multibody_sim_w_control import X_WObject
print "X_WBottle, ground truth:\n", X_WObject
print "X_WBottle, estiated by ICP\n", np.linalg.inv(X_MS_bottle)

# Display the transfomed scene points in yellow in meshcat.
VisualizeTransformedScenePointCloud(X_MS_bottle, scene_point_cloud_cropped[indices_bottle], vis)

### 1.2 Estimating the pose of the green box in the world frame using ICP. 

First, let's load the model points of the box. The model point cloud is generated by uniformly sampling points on the faces of the box. 

In [ ]:
num_points = 10000
box_point_cloud_model = GetBoxModelPointCloud(num_points=num_points)

# clear visualizer
vis.delete()

# show point cloud in meshcat
vis['model_point_cloud'].set_object(
    g.PointCloud(box_point_cloud_model.T, make_meshcat_color_array(num_points, 0.5, 0, 0), size=0.002))

We'll take a shortcut with segmentation this time.

In [ ]:
# Segment box points from scene
box_point_cloud_scene = SegmentBox(scene_point_cloud)

# show point cloud in meshcat
vis["scene_point_cloud"].set_object(
    g.PointCloud(position=box_point_cloud_scene.T,
                 color=make_meshcat_color_array(box_point_cloud_scene.shape[0], 0, 0.5, 0),
                 size=0.002))

Notice that the segmented box does not have any points on its bottom face (because no camera can see it). 
Trying to run ICP between the point cloud of a bottomless box and a point cloud consisting of points 
uniformly sampled from the surface of the box would probably generate wrong results. 

The following cell runs ICP on the box for 10 times. Every time, a new model point cloud is generated by randomly sampling a new set of points.

You can watch the estimated poses displayed in meshcat. We're trying to show how brittle ICP can be in this case: the success rate is only about 50% at its best, even though all sampled point clouds look similar.

In [ ]:
# get ground truth box pose.
from kuka_multibody_sim_w_control import X_WBox
R_WB = X_WBox.rotation()
p_WBo = X_WBox.translation()


success_count = 0
for i in range(10):
    i += 1
    box_point_cloud_model = GetBoxModelPointCloud(num_points=10000)
    X_MS_box, mean_error, num_iters = \
        icp(box_point_cloud_scene, box_point_cloud_model,
            init_guess=np.eye(4), max_iterations=100, tolerance=1e-5)
    X_WB = np.linalg.inv(X_MS_box)
    R_WB_estimated = X_WB[0:3, 0:3]
    p_WBo_estimated = X_WB[0:3, 3]

    is_rotation_close = np.abs(R_WB - R_WB_estimated).max() < 0.02
    is_position_close = np.abs(p_WBo - p_WBo_estimated).max() < 0.02
    
    is_successful = is_position_close and is_rotation_close
    if is_successful:
        success_count += 1
    
    print "trial ", i, 'success' if is_successful else 'fail'
        
    # Create a yellow meshcat point cloud for visualization.
    VisualizeTransformedScenePointCloud(X_MS_box, box_point_cloud_scene, vis)

print "success_count:", success_count

Why does ICP work unreliably in this case? First of all, the scene point cloud is incomplete: it doesn't have a bottom face. In addition, due to the positioning of cameras, the distribution of points on different faces of the scene point cloud is not uniform: there are a lot more points on the top face.

As discussed in the lectures, you could try normal ICP, or formulating pose estimation as an inverse kinematics problem with constraints. Fill in function `EstimateBoxPoseSmartly` in `point_cloud_processing.py` with your improved strategy. 

**This task is optional.** Bonus points will be given if your strategy can succeed more than 8 out of 10 times. 

## 2. Trajectory planning tasks
Let's define a few things before laying out the requirements. 
- Q is a point fixed to the end effector frame. 
- `p_EQ = [0, 0.12, 0]`, where `E` is the end effector frame and `Q` represents point Q, denotes the fixed position of Q in frame E.
- `p_WQ`, where `W` is the world frame and `Q` represents point Q, denotes the position of Q in frame `W`.


Executing trajectories provided to you in function `GenerateIiwaTrajectoriesAndGripperSetPoints` (in file `kuka_pick_and_place_trajectory_generation.py`), you'll find that robot arm 
- starts at the upright posture (all joint angles at 0), and then 
- moves point Q to `p_WQ_home = [0.6, 0, 1.1]` and stays there, and finally
- opens the gripper. 


Your ask is to pick up from where the robot stops. Complete function `GenerateIiwaTrajectoriesAndGripperSetPoints`. **The completed function should return a list (python list) of trajectories (`pydrake.trajectories.PiecewisePolynomial`) that picks up the bottle and places it on the green box**. 

Each trajectory has a duration that the simulator is aware of. The simulator makes sure that all trajectories are executed, and simulates for an additional 5 seconds for velocities to damp out. At the end of the simulation, your system (robot + gripper + bottle + box) needs to satisfy the following requirements:
 
1. point Q in the end effector frame `E` has returned to `p_WQ_home`;
3. the box should not move away from its initial pose;
4. everything should be have close-to-zero velocity (doesn't look like it's moving in the visualizer); 
4. the gripper must be open (we'll explain what open means soon);
5. the bottle is on the top of the box;
6. the origin of the bottle's body frame (which is roughly the center of the bottom of the bottle) needs to be within the top face of the box. 
7. (This is not needed to pass the test, but highly recommended.) Keep `R_WEa`, the orientation of frame Ea relative to the world frame, at a fixed roll-pitch-yaw angle: (r=0, p=pi/4, y=0). Frame Ea, or End_effector_world_aligned, is defined at the beginning of `kuka_multibody_sim_w_control.py`. (r=0, p=pi/4, y=0) is the orientation of the gripper at the end of the trajectory provided to you. 
    You could grasp the bottle at whatever angle you like, but changing the orientation of the gripper involves interpolating rotations, which is nontrivial. Unless you are willing to write that yourself, it's easier to just keep the end effector orientation constant. 

The gripper fingers and body are connected by translational joints. Setting `gripper_setpoint` to 0.05 (meters) means the gripper is wide open, setting it to 0 means the gripper is closed. 


### Hints and tips
#### Drake utilities that will probably come in handy
- You'll probably make heavy use of `pydrake.trajectories.PiecewisePolynomial`. The C++ documentation can be found [here](http://drake.mit.edu/doxygen_cxx/classdrake_1_1trajectories_1_1_piecewise_polynomial.html#acd5823d88e611200dc63daaf05eeb9d9). Examples on how to use the python bindings can be found [here](https://github.com/RobotLocomotion/drake/blob/f45616e2537f1b60581f1b47964c88631af98321/bindings/pydrake/test/trajectories_test.py). 
- `pydrake.math` has convenient methods for the creation and conversion of converting rotation representations. We've used [`pydrake.math.RotationMatrix`](http://drake.mit.edu/doxygen_cxx/classdrake_1_1math_1_1_rotation_matrix.html) and [`pydrake.math.RollPitchYaw`](http://drake.mit.edu/doxygen_cxx/classdrake_1_1math_1_1_roll_pitch_yaw.html). 

- Pydrake also provides python bindings for many [Eigen](http://eigen.tuxfamily.org/index.php?title=Main_Page) types. For example, we've used `Isometry3` in this pset, which is a convenient wrapper around 4x4 uniform transformation matrices that gives you easy access to the rotational and translational components with methods like `Isometry3.rotation()` or `Isometry3.set_translation()`. 



#### General tips on finding documentation/examples in Drake
Every function in Drake is called at least once in Drake's unit tests. Therefore, if you want to look for examples on how to use a specific function/class, you can go to Drake's [github repo](https://github.com/RobotLocomotion/drake), search for the function/class of interest, and look only at python code. Drake's [doxygen](http://drake.mit.edu/doxygen_cxx/classdrake_1_1trajectories_1_1_piecewise_polynomial.html#acd5823d88e611200dc63daaf05eeb9d9) is also a useful reference. 


#### Task space interpolation with IK
Solving IK gives you points in the joint space. Interpolating between points that are "far away" in joint space could create very curvilinear end effector trajectories in task space. This could lead to undesirable results, such as knocking stuff off the table. 

To make sure that end effector moves more or less along a straight line in task space, we recommend you implement the following method.

```python
    '''
    This is pseudo code!
    p_WQ_start is the starting position of point Q (the point Q we defined earlier!)
    p_WQ_start is the end position of point Q.
    duration is the duration of the trajectory measured in seconds. 
    R_WE is the a fixed orientation that we want the end effector frame to keep along the trajectory.
    num_knots is the number of knot points in the trajectory. Generally speaking,the larger num_knots is, the smoother the trajectory. 
    '''
def MoveEndEffectorAlongStraightLine(p_WQ_start, p_WQ_end, duration, R_WE, q_initial_guess, num_knots):
    t_knots = np.linspace(0, duration, num_knots+1)
    q_knots = np.zeros((num_knots+1, plant.num_positions()))
    q_knots[0] = q_initial_guess
    
    for i in range(num_knots):
        ik = pydrake.multibody.inverse_kinematics.InverseKinematics(plant)
        ik.AddPositionConstraint() # interpolate between p_WQ_start and p_WQ_end
        ik.AddOrientationConstraint(R_WE)
        ik.SetInitialGuess(q_knots[i]) # this is very important!
        ik.Solve()
        prog = ik.get_mutable_prog()
        q_knots[i+1] = prog.GetSolution(ik.q())
    
    return PiecewisePolynomia.Cubic(q_knots) # return a cubic spline that connects all q_knots.    
```

#### Don't grab the bottle by its cap
It's not only because it would score poorly in some grasp metrics. We believe that contact detection between meshes does not give trustworthy contact forces. As a result, contact geometries in `MultiBodyTree` are made of elementary shapes such as spheres, boxes and cylinders. The contact geometry of the bottle is a box that's slightly smaller than the bottle. So the cap of the bottle is transparent as far as contact is concerned. 

In [ ]:
from kuka_multibody_sim_w_control import ForwardKinematicsMbp

# get ground truth poses of object and box from results in part 1.
from kuka_multibody_sim_w_control import X_WObject, X_WBox

# clean up visualizer
vis['model_point_cloud'].delete()
vis['scene_point_cloud'].delete()
vis['transformed_scene'].delete()

# create multibodyplant and scene graph for simulation.
plant, scene_graph, model_list = CreateMultiBodyPlant()

# Note that X_WObject, X_WBox are not numpy arrays, their type is pydrake.util.eigen_geometry.Isometry3
q_traj_list, gripper_setpoint_list = GenerateIiwaTrajectoriesAndGripperSetPoints(plant, model_list, X_WObject, X_WBox)

# q_traj_list is a list of PiecewisePolynomials. The trajectories in the list will be executed in sequence. 
# gripper_setpoint_list is a list of doubles. 0.05 means the gripper is open, 0 means it's closed.
# q_traj_list must have as many elements as gripper_setpoint_list: for every robot arm trajectory, 
# there is a gripper set point. 
state_log = RunSimulation(plant, scene_graph, q_traj_list, gripper_setpoint_list)

# Calculate final end effector pose.
x_final = state_log.data()[:,-1].flatten()
X_WE = ForwardKinematicsMbp(x_final, plant.GetFrameByName("body"), plant)
print "Final end effector pose in world frame\n", X_WE

# Calculate position of point Q in world frame.
p_EQ = np.array([0., 0.12, 0.])
print "Final postion of point Q in world frame\n", X_WE.multiply(p_EQ)

# Get final velocity of the system
v = x_final[plant.num_positions():]
print "maximum component of v: ", np.abs(v).max()

# Run Tests
Running the cell below will print out a score for every test and the total score.

In [ ]:
import os
# Run the tests
os.popen("python test_pset_2.py test_results.json")

# Print the results json for review
import test_pset_2
print test_pset_2.pretty_format_json_results("test_results.json")